In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# 1. Import data

In [2]:
path = r'C:\Users\Yan Peng\10-2020 Instacart Basket Analysis'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_derived.pkl'))

In [4]:
ords_prods_merge.shape

(32435059, 19)

# 2. Creating customer loyalty flag

In [5]:
# Add a new column containing the total count of orders for each customer

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
# Check transformed dataframe

pd.options.display.max_rows = None
ords_prods_merge.head(100)

In [7]:
# Create a flag that assigns loyalty labels based on customer's max order value

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [8]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [9]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
# Count number of orders within each label

ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15891507
Loyal customer      10294027
New customer         6249525
Name: count, dtype: int64

In [ ]:
# Check updated dataframe

ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(300)

# 3. Product prices by loyalty label

In [13]:
# Perform multiple aggregations on "prices" column grouped by "loyalty_flag"

ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

##### New customers exhibit the highest average product prices, followed by regular customers. Conversely, loyal customers tend to order products with the lowest average prices. The maximum values appear to be outliers.

# 4. Creating customer spending flag

In [15]:
# Add a new column containing average item prices purchased by customers

ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [ ]:
# Check transformed dataframe

ords_prods_merge.head(100)

In [16]:
# Create a flag that distinguishes customers based on their average spending 

ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'

In [17]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

In [18]:
# Count number of orders within each category

ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31800381
High spender      634678
Name: count, dtype: int64

# 5. Creating order frequency flag

In [19]:
# Add a new column with median values of number of days since prior order for all customers

ords_prods_merge['median_days_prior'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [ ]:
# Check transformed dataframe

ords_prods_merge.head(100)

In [21]:
# Create a flag that distinguishes customers based on their order frequency

ords_prods_merge.loc[ords_prods_merge['median_days_prior'] > 20, 'order_freq_flag'] = 'Non-frequent customer'

In [22]:
ords_prods_merge.loc[(ords_prods_merge['median_days_prior'] > 10) & (ords_prods_merge['median_days_prior'] <= 20) , 'order_freq_flag'] = 'Regular customer'

In [23]:
ords_prods_merge.loc[ords_prods_merge['median_days_prior'] <= 10, 'order_freq_flag'] = 'Frequent customer'

In [24]:
# Count number of orders within each category

ords_prods_merge['order_freq_flag'].value_counts(dropna = False)

order_freq_flag
Frequent customer        21578273
Regular customer          7217066
Non-frequent customer     3639720
Name: count, dtype: int64

# 7. Export data

In [25]:
# Check dataframe before export

ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,price_label,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_days_prior,order_freq_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2539329,1,1,2,8,NaN,14084,2,0,both,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2539329,1,1,2,8,NaN,12427,3,0,both,...,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2539329,1,1,2,8,NaN,26088,4,0,both,...,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,2539329,1,1,2,8,NaN,26405,5,0,both,...,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [26]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_flagged.pkl'))